# Analysis of AIDA impact simulations

## 0) Imports

In [14]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import h5py
import scipy

%load_ext nb_black
%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 10)
pd.set_option("display.precision", 2)

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

## 1) Load simulation data:

In [2]:
dirpath = "../data/"
filename = "impact_damage_visc1.0089.h5"

<IPython.core.display.Javascript object>

In [3]:
with h5py.File(f"{dirpath}{filename}", "r") as hdf:
    keys = list(hdf.keys())

<IPython.core.display.Javascript object>

In [4]:
keys

['DIM_root_of_damage_porjutzi',
 'DIM_root_of_damage_tensile',
 'a',
 'a_grav',
 'activation_thresholds',
 'alpha_jutzi',
 'dalphadt',
 'damage_total',
 'ddamage_porjutzidt',
 'dddt',
 'ddeviatoric_stress_dt',
 'dedt',
 'deviatoric_stress',
 'drhodt',
 'e',
 'local_strain',
 'm',
 'material_type',
 'maximum_number_of_flaws',
 'number_of_activated_flaws',
 'number_of_interactions',
 'p',
 'rho',
 'sml',
 'soundspeed',
 'time',
 'tree_depth',
 'v',
 'x']

<IPython.core.display.Javascript object>

In [5]:
with h5py.File(f"{dirpath}{filename}", "r") as hdf:
    positions = hdf.get("x")[:]
    velocities = hdf.get("v")[:]

<IPython.core.display.Javascript object>

In [16]:
df = pd.DataFrame(
    {
        "x": positions[:, 0],
        "y": positions[:, 1],
        "z": positions[:, 2],
        "vx": velocities[:, 0],
        "vy": velocities[:, 1],
        "vz": velocities[:, 2],
    }
)

<IPython.core.display.Javascript object>

In [17]:
df

,x,y,z,vx,vy,vz
0,-0.48,-0.30,-9.75,-0.02,-1.61e-02,-0.55
1,0.13,-0.30,-9.75,0.01,-1.60e-02,-0.55
2,-0.48,0.07,-9.75,-0.02,6.37e-03,-0.55
3,0.13,0.07,-9.75,0.01,6.14e-03,-0.55
4,-0.48,0.44,-9.75,-0.02,2.70e-02,-0.54
...,...,...,...,...,...,...
99740,3.41,-1.00,5.16,471.58,-1.29e+02,798.59
99741,0.20,0.71,-1.08,10.65,-1.24e+02,8.84
99742,3.36,1.31,5.29,465.09,1.90e+02,819.28
99743,-0.22,3.70,7.12,30.25,4.91e+02,1119.42


<IPython.core.display.Javascript object>

## 2) Compute beta factor

###  2.0) material properties

In [19]:
M = 3 * 10 ** 9
r_surface = 80
G = scipy.constants.G

<IPython.core.display.Javascript object>

### 2.1) Find particles with vz > 0 above escape velocity

In [20]:
df["escape"] = 0.5 * (
    df.vx.values ** 2 + df.vy.values ** 2 + df.vz.values ** 2
) - G * M / (r_surface + df.z.values)

<IPython.core.display.Javascript object>

In [40]:
df[(df.vz > 0) & (df.escape > 0)]

,x,y,z,vx,vy,vz,escape
25801,-2.00,-8.84,-3.48,-7.38e-02,-0.34,5.48e-03,0.06
25802,-1.39,-8.84,-3.48,-5.23e-02,-0.36,7.23e-03,0.06
25803,-0.78,-8.84,-3.48,-2.79e-02,-0.37,6.78e-03,0.07
25804,-0.18,-8.84,-3.48,-2.99e-03,-0.38,6.51e-03,0.07
25805,0.43,-8.84,-3.48,2.41e-02,-0.37,6.87e-03,0.07
...,...,...,...,...,...,...,...
99740,3.41,-1.00,5.16,4.72e+02,-128.63,7.99e+02,438343.30
99741,0.20,0.71,-1.08,1.07e+01,-123.85,8.84e+00,7764.72
99742,3.36,1.31,5.29,4.65e+02,189.76,8.19e+02,461771.88
99743,-0.22,3.70,7.12,3.03e+01,490.92,1.12e+03,747505.30


<IPython.core.display.Javascript object>